# Axisymmetric formulation for elastic structures of revolution

In this numerical tour, we will deal with axisymmetric problems of elastic solids. We will consider a solid of revolution around a fixed axis $(Oz)$, the loading, boundary conditions and material properties being also invariant with respect to a rotation along the symmetry axis. The solid cross-section in a plane $\theta=\text{cst}$ will be represented by a two-dimensional domain $\omega$ for which the first spatial variable (`x[0]` in FEniCS) will represent the radial coordinate $r$ whereas the second spatial variable will denote the axial variable $z$.

## Problem position

We will investigate here the case of a hollow hemisphere of inner (resp. outer) radius $R_i$ (resp. $R_e$). Due to the revolution symmetry, the 2D cross-section corresponds to a quarter of a hollow cylinder.

In [14]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
from dolfinx.fem.petsc import LinearProblem
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot

from __future__ import print_function

from dolfinx.io import XDMFFile, gmshio
from dolfinx.mesh import meshtags_from_entities

import matplotlib.pyplot as plt

In [15]:
# Mesh
try:
    import gmsh
except ImportError:
    print("This demo requires gmsh to be installed")
    exit(0)

Re = 11.
Ri = 9.

def gmsh_hollow_circle(model: gmsh.model, name: str, option: gmsh.option) -> gmsh.model:
    model.add(name)
    model.setCurrent(name)

    # Create outer and inner circles
    outer_circle = model.occ.addDisk(0, 0, 0, Re, Re)
    inner_circle = model.occ.addDisk(0, 0, 0, Ri, Ri)
    
    # Cut the inner circle from the outer circle to create a hollow circle
    hollow_circle = model.occ.cut([(2, outer_circle)], [(2, inner_circle)])[0]
    model.occ.synchronize()

    # Add physical groups
    gdim = 2
    model.addPhysicalGroup(gdim, [hollow_circle[0][1]], tag=1)
    model.setPhysicalName(2, 1, "Hollow circle")
    
    boundary_entities = model.getEntities(2)
    boundary_ids = [entity[1] for entity in boundary_entities if entity[1] != hollow_circle[0][1]]
    model.addPhysicalGroup(2, boundary_ids, tag=2)
    model.setPhysicalName(2, 2, "Boundary")

    option.setNumber("Mesh.CharacteristicLengthMin", 0.5)
    option.setNumber("Mesh.CharacteristicLengthMax", 0.5)
    model.mesh.generate(gdim)
    return model

def create_mesh(comm: MPI.Comm, model: gmsh.model, name: str, filename: str, mode: str):
    msh, ct, ft = gmshio.model_to_mesh(model, comm, rank=0)
    msh.name = name
    ct.name = f"{msh.name}_cells"
    ft.name = f"{msh.name}_facets"

    tdim = msh.topology.dim
    fdim = tdim - 1

    with XDMFFile(msh.comm, filename, mode) as file:
        msh.topology.create_connectivity(fdim, tdim)
        file.write_mesh(msh)
        file.write_meshtags(
            ct, msh.geometry, geometry_xpath=f"/Xdmf/Domain/Grid[@Name='{msh.name}']/Geometry"
        )
        file.write_meshtags(
            ft, msh.geometry, geometry_xpath=f"/Xdmf/Domain/Grid[@Name='{msh.name}']/Geometry"
        )

# Initialize Gmsh
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 0)

# Create model
model = gmsh.model()
option = gmsh.option()

# Create a Gmsh model
model = gmsh_hollow_circle(model, "HollowCircle", option)
model.setCurrent("HollowCircle")

# Create a DOLFINx mesh from the Gmsh model and save to an XDMF file
create_mesh(MPI.COMM_SELF, model, "hollow_circle", "out_gmsh/hollow_circle.xdmf", "w")

# We can use this to clear all the model data:
# gmsh.clear()
# gmsh.finalize()

In [16]:
# Domain
gdim = 2
gmsh_model_rank = 0
mesh_comm = MPI.COMM_WORLD

domain, cell_markers, facet_markers = gmshio.model_to_mesh(gmsh.model, mesh_comm, gmsh_model_rank, gdim=gdim)

In [17]:
# Determining facets to apply boundary conditions
def innerRadius(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2), Ri)

def outerRadius(x):
    return np.isclose(np.sqrt(x[0]**2 + x[1]**2), Re)

fdim = domain.topology.dim - 1
print(f"Domain topology = {domain.topology.dim}")

inner_facets = mesh.locate_entities_boundary(domain, fdim, innerRadius)
outer_facets = mesh.locate_entities_boundary(domain, fdim, outerRadius)

# Markers
marked_facets = np.hstack([inner_facets, outer_facets])
marked_values = np.hstack([np.full_like(inner_facets, 1), np.full_like(outer_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

print(f"Boundary conditions = {u_bc}")

V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

inner_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
outer_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(2))

print(f"inner_dofs = {inner_dofs}")
print(f"outer_dofs = {outer_dofs}")

# bcs = [fem.dirichletbc(u_bc, left_dofs, V), fem.dirichletbc(u_bc, right_dofs, V)]
bcs = [fem.dirichletbc(u_bc, inner_dofs, V)]

Domain topology = 2
Boundary conditions = [0. 0.]
inner_dofs = [  70   92   94   95   96  134  135  143  145  172  173  191  193  217
  218  235  237  265  266  282  284  309  310  326  328  359  360  373
  375  404  405  415  417  451  452  462  464  499  500  510  512  544
  545  557  559  591  592  601  603  636  637  643  645  686  687  693
  695  731  732  738  740  778  779  785  787  826  827  833  835  876
  877  883  885  921  922  928  930  965  966  972  974 1012 1013 1019
 1021 1054 1055 1061 1063 1104 1105 1111 1113 1149 1150 1156 1158 1193
 1194 1200 1202 1243 1244 1250 1252 1294 1295 1301 1303 1339 1340 1346
 1348 1383 1384 1390 1392 1427 1428 1434 1436 1474 1475 1481 1483 1527
 1528 1534 1536 1573 1574 1580 1582 1617 1618 1624 1626 1661 1662 1668
 1670 1705 1706 1712 1714 1751 1752 1759 1761 1795 1796 1804 1806 1841
 1842 1848 1850 1885 1886 1892 1894 1935 1936 1942 1944 1983 1984 1990
 1992 2030 2031 2037 2039 2075 2076 2082 2084 2119 2120 2126 2128 2169
 2170 2176 217

In [18]:
B = fem.Constant(domain, default_scalar_type((0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0)))

v = ufl.TestFunction(V)
u = fem.Function(V)

In [19]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
G = mu
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))

# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2

# Stress
# Hyper-elasticity
# P = ufl.diff(psi, F)
# Elasticity
P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)

# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

problem = NonlinearProblem(F, u, bcs)

solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


## Definition of axisymmetric strains

For axisymmetric conditions, the unkown displacement field is of the form:

$$\begin{equation}
\boldsymbol{u} = u_r(r,z)\boldsymbol{e}_r + u_z(r,z)\boldsymbol{e}_z
\end{equation}$$

As a result, we will work with a standard VectorFunctionSpace of dimension 2. The associated strain components are however given by:

$$\begin{equation}
\boldsymbol{\varepsilon} = \begin{bmatrix} \partial_r u_r & 0 & (\partial_z u_r + \partial_r u_z)/2 \\ 
0 & u_r/r & 0 \\
(\partial_z u_r + \partial_r u_z)/2 & 0 & \partial_z u_z\end{bmatrix}_{(\boldsymbol{e}_r,\boldsymbol{e}_\theta,\boldsymbol{e}_z)}
\end{equation}$$

The previous relation involves explicitly the radial variable $r$, which can be obtained from the SpatialCoordinate `x[0]`, the strain-displacement relation is then defined explicitly in the `eps` function.

> **Note**: we could also express the strain components in the form of a vector of size 4 in alternative of the 3D tensor representation implemented below.

## Resolution

The rest of the formulation is similar to the 2D elastic case with a small difference in the integration measure. Indeed, the virtual work principle reads as:

$$\begin{equation}
 \text{Find } \boldsymbol{u}\in V \text{ s.t. } \int_{\Omega}
 \boldsymbol{\sigma}(\boldsymbol{u}):\boldsymbol{\varepsilon}(\boldsymbol{v}) d\Omega
 = \int_{\partial \Omega_T} \boldsymbol{T}\cdot\boldsymbol{v}  dS \quad \forall\boldsymbol{v} \in V
 \end{equation}$$
 
where $\boldsymbol{T}$ is the imposed traction on some part $\partial \Omega_T$ of the domain boundary.

In axisymmetric conditions, the full 3D domain $\Omega$ can be decomposed as $\Omega = \omega \times [0;2\pi]$ where the interval represents the $\theta$ variable. The integration measures therefore reduce to $d\Omega = d\omega\cdot(rd\theta)$ and $dS = ds\cdot(rd\theta)$ where $dS$ is the surface integration measure on the 3D domain $\Omega$ and $ds$ its counterpart on the cross-section boundary $\partial \omega$. Exploiting the invariance of all fields with respect to $\theta$, the previous virtual work principle is reformulated on the cross-section only as follows:

$$\begin{equation}
 \text{Find } \boldsymbol{u}\in V \text{ s.t. } \int_{\omega}
 \boldsymbol{\sigma}(\boldsymbol{u}):\boldsymbol{\varepsilon}(\boldsymbol{v}) rd\omega
 = \int_{\partial \omega_T} \boldsymbol{T}\cdot\boldsymbol{v}  rds \quad \forall\boldsymbol{v} \in V
 \end{equation}$$
 
where the $2\pi$ constants arising from the integration on $\theta$ have been cancelled on both sides. As a result, the bilinear and linear form are similar to the plane 2D case with the exception of the additional $r$ term in the integration measures.

The final formulation is therefore pretty straightforward. Since a uniform pressure loading is applied on the outer boundary, we will also need the exterior normal vector to define the work of external forces form.

First, smooth contact conditions are assumed on both $r=0$ (`Left`) and $z=0$ (`Bottom`) boundaries. For this specific case, the solution corresponds to the classical hollow sphere under external pressure with a purely radial displacement:

$$\begin{equation}
u_r(r) = -\dfrac{R_e^3}{R_e^3-R_i^3}\left((1 − 2\nu)r + (1 + \nu)\dfrac{R_i^3}{2r^2}\right)\dfrac{p}{E},
\quad u_z=0
\end{equation}$$

The second loading case corresponds to a fully clamped condition on $z=0$, the vertical boundary remaining in smooth contact.

In [20]:
pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif("deformation.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

In [21]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5

for n in range(1, 10):
    T.value[1] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()

2024-07-27 20:14:45.274 (  58.751s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 20:14:45.296 (  58.774s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 20:14:45.304 (  58.781s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1.34213e-16 (tol = 1e-08) r (rel) = 5.34292e-15(tol = 1e-08)
2024-07-27 20:14:45.304 (  58.781s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.
2024-07-27 20:14:45.444 (  58.921s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 20:14:45.457 (  58.934s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 20:14:45.462 (  58.939s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (a

Time step 1, Number of iterations 2, Load [ 0.  -1.5]
Time step 2, Number of iterations 2, Load [ 0. -3.]
Time step 3, Number of iterations 2, Load [ 0.  -4.5]
Time step 4, Number of iterations 2, Load [ 0. -6.]
Time step 5, Number of iterations 2, Load [ 0.  -7.5]


2024-07-27 20:14:45.665 (  59.143s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 20:14:45.675 (  59.153s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 20:14:45.682 (  59.159s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (abs) = 1.28395e-16 (tol = 1e-08) r (rel) = 5.11129e-15(tol = 1e-08)
2024-07-27 20:14:45.682 (  59.159s) [main            ]       NewtonSolver.cpp:252   INFO| Newton solver finished in 2 iterations and 2 linear solver iterations.
2024-07-27 20:14:45.718 (  59.195s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 20:14:45.728 (  59.205s) [main            ]              petsc.cpp:700   INFO| PETSc Krylov solver starting to solve system.
2024-07-27 20:14:45.734 (  59.211s) [main            ]       NewtonSolver.cpp:38    INFO| Newton iteration 2: r (a

Time step 6, Number of iterations 2, Load [ 0. -9.]
Time step 7, Number of iterations 2, Load [  0.  -10.5]
Time step 8, Number of iterations 2, Load [  0. -12.]
Time step 9, Number of iterations 2, Load [  0.  -13.5]
